In [478]:
pip install langchain langgraph


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [479]:
pip install langchain-google-genai

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [480]:
from dotenv import load_dotenv
load_dotenv()

from langchain.chat_models import init_chat_model
from langgraph.graph import StateGraph, START, END
from typing import Annotated
from typing_extensions import TypedDict
from langgraph.graph.message import add_messages
from langgraph.types import interrupt, Command
from langgraph.checkpoint.memory import MemorySaver

In [481]:
class workflowState(TypedDict):
    preferences: list[str]
    breakfast: list[str]
    lunch: list[str]
    dinner: list[str]

In [482]:
llm = init_chat_model("google_genai:gemini-2.0-flash")

In [483]:
def get_preferences(state: workflowState):
    state["preferences"] = "vegetarian"
    return { "preferences":  state["preferences"] }

In [ ]:
def gen_breakfast(state: workflowState):
    # result = llm.invoke(f"generate {state["preferences"]} breakfast meal coma seperated small list")
    state["breakfast"] = ["pancakes", "omelette", "fruit salad"]
    return { "breakfast":  state["breakfast"] }

In [ ]:
def gen_lunch(state: workflowState):
    # result = llm.invoke(f"generate {state["preferences"]} lunch meal coma seperated small list")
    state["lunch"] = ["salad", "sandwich", "sushi"]
    return { "lunch":  state["lunch"] }

In [ ]:
def gen_dinner(state: workflowState):
    # result = llm.invoke(f"generate {state["preferences"]} dinner meal coma seperated small list")
    state["dinner"] = ["pasta", "stir-fry", "tacos"]
    return Command(
        goto='review',
        update={ "dinner":  state["dinner"] }
    )
    

In [487]:
def review(state: workflowState):
    print("\n Generated meal plan")
    print("Breakfast:", state["breakfast"])
    print("Lunch:", state["lunch"])
    print("Dinner:", state["dinner"])

    data = f"Generated mean plan\n Breakfast: {state["breakfast"]}, Lunch: {state["lunch"]}, Dinner: {state["dinner"]} review and approve the meal"
    decision = interrupt(data)
    if decision == "yes":
        return Command(
            goto="finalize"
        )
    else:
        nodevalue = input('Enter the step you want to go')
        return Command(
            goto=nodevalue
        )

    

In [488]:
def finalize(state: workflowState):
    print(f"meal finalized-->   Breakfast: {state["breakfast"]}, Lunch: {state["lunch"]}, Dinner: {state["dinner"]}")
    return state

In [489]:
graph = StateGraph(workflowState)

memory = MemorySaver()
graph.add_node("get_preferences", get_preferences)
graph.add_node("gen_breakfast", gen_breakfast)
graph.add_node("gen_lunch", gen_lunch)
graph.add_node("gen_dinner", gen_dinner)
graph.add_node("review", review)
graph.add_node("finalize", finalize)
# graph.add_node("reject", reject)

graph.add_edge(START, "get_preferences")
graph.add_edge("get_preferences", "gen_breakfast")
graph.add_edge("get_preferences", "gen_lunch")
graph.add_edge("get_preferences", "gen_dinner")

graph.add_edge("gen_breakfast", "review")
graph.add_edge("gen_lunch", "review")
graph.add_edge("gen_dinner", "review")

graph.add_edge("review", "finalize")
graph.add_edge("finalize", END)

workflow = graph.compile(checkpointer=memory)

In [490]:

config = {"configurable": {"thread_id": "1"}}
result = workflow.invoke({}, config=config, stream_mode="updates")
result


 Generated meal plan
Breakfast: Omelette, Pancakes, Waffles, French Toast, Breakfast Burrito, Oatmeal, Yogurt Parfait, Avocado Toast, Smoothie, Tofu Scramble
Lunch: result.content
Dinner: result.content


[{'get_preferences': {'preferences': 'vegetarian'}},
 {'gen_dinner': {'dinner': 'result.content'}},
 {'gen_lunch': {'lunch': 'result.content'}},
 {'gen_breakfast': {'breakfast': 'Omelette, Pancakes, Waffles, French Toast, Breakfast Burrito, Oatmeal, Yogurt Parfait, Avocado Toast, Smoothie, Tofu Scramble'}},
 {'__interrupt__': (Interrupt(value='Generated mean plan\n Breakfast: Omelette, Pancakes, Waffles, French Toast, Breakfast Burrito, Oatmeal, Yogurt Parfait, Avocado Toast, Smoothie, Tofu Scramble, Lunch: result.content, Dinner: result.content review and approve the meal', resumable=True, ns=['review:7cc96a91-063d-68c9-7ee8-b5ba4385817d']),)}]

In [491]:
snapshot = workflow.get_state(config)
snapshot.next

('review',)

In [ ]:
for chunk in workflow.stream({}, config=config, stream_mode="updates"):
    for node_id, value in chunk.items():
        print(f"Node ID: {node_id}, Value: {value} ---------------")
        if(node_id == "__interrupt__"):
            while True: 
                decision = input("Approve meal plan? (yes/no): ").strip().lower()

                if decision.lower() == "yes":
                    result = workflow.invoke(Command(resume=decision), config=config, stream_mode="updates")
                    print(result)
                    break
                else:
                    value = input('Eneter the node name you want to go')
                    result = workflow.invoke(Command(goto=value), config=config, stream_mode="updates")
                    print(result)

Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised ResourceExhausted: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. [violations {
  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_requests"
  quota_id: "GenerateRequestsPerDayPerProjectPerModel-FreeTier"
  quota_dimensions {
    key: "model"
    value: "gemini-2.0-flash"
  }
  quota_dimensions {
    key: "location"
    value: "global"
  }
  quota_value: 200
}
, links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, retry_delay {
  seconds: 37
}
].



 Generated meal plan
Breakfast: Omelet, Pancakes, Waffles, French Toast, Breakfast Burrito, Oatmeal, Yogurt Parfait, Avocado Toast, Breakfast Hash, Smoothie
Lunch: result.content
Dinner: result.content

 Generated meal plan
Breakfast: Omelet, Pancakes, Waffles, French Toast, Breakfast Burrito, Oatmeal, Yogurt Parfait, Avocado Toast, Breakfast Hash, Smoothie
Lunch: result.content
Dinner: result.content
meal finalized-->   Breakfast: Omelet, Pancakes, Waffles, French Toast, Breakfast Burrito, Oatmeal, Yogurt Parfait, Avocado Toast, Breakfast Hash, Smoothie, Lunch: result.content, Dinner: result.content
[{'review': None}, {'finalize': {'preferences': 'vegetarian', 'breakfast': 'Omelet, Pancakes, Waffles, French Toast, Breakfast Burrito, Oatmeal, Yogurt Parfait, Avocado Toast, Breakfast Hash, Smoothie', 'lunch': 'result.content', 'dinner': 'result.content'}}]
